In [1]:
!pip install datasets scikit-learn joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [2]:
from datasets import load_dataset
import pandas as pd

# Load Banking77
dataset = load_dataset("PolyAI/banking77")

# intent names
labels = dataset["train"].features["label"].names


df = pd.DataFrame({
    "text": [x["text"] for x in dataset["train"]],
    "intent": [labels[x["label"]] for x in dataset["train"]]
})

df.head()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.78k [00:00<?, ?B/s]

banking77.py:   0%|          | 0.00/7.17k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.89k [00:00<?, ?B/s]

The repository for PolyAI/banking77 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/banking77.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] trust_remote_code=True
The repository for PolyAI/banking77 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/PolyAI/banking77.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

,text,intent
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

# Split
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["intent"], test_size=0.2, random_state=42)

# pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000))
])

# Train
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


                                                  precision    recall  f1-score   support

                           Refund_not_showing_up       0.91      0.94      0.92        32
                                activate_my_card       0.97      0.90      0.93        31
                                       age_limit       0.96      0.96      0.96        25
                         apple_pay_or_google_pay       0.96      1.00      0.98        23
                                     atm_support       0.94      0.89      0.92        19
                                automatic_top_up       1.00      0.96      0.98        27
         balance_not_updated_after_bank_transfer       0.79      0.84      0.81        31
balance_not_updated_after_cheque_or_cash_deposit       0.95      0.88      0.91        41
                         beneficiary_not_allowed       0.83      1.00      0.91        24
                                 cancel_transfer       0.86      0.89      0.87        35
         

In [4]:
from google.colab import drive
import joblib

# Mount Google Drive
drive.mount('/content/drive')

# Save model
joblib.dump(pipeline, '/content/drive/MyDrive/BankingData/banking77_model.joblib')
print("✅ Model saved to your Google Drive.")


Mounted at /content/drive
✅ Model saved to your Google Drive.


In [6]:
# Load the model
model = joblib.load('/content/drive/MyDrive/BankingData/banking77_model.joblib')


queries = [
    "I lost my card",
    "How can I get a new credit card?",
    "Can I increase my overdraft limit?",
    "I'd like to cancel a payment",
    "Tell me your working hours"
]

for q in queries:
    intent = model.predict([q])[0]
    print(f"❓ Query: {q}\n➡️ Predicted intent: {intent}\n")


❓ Query: I lost my card
➡️ Predicted intent: lost_or_stolen_card

❓ Query: How can I get a new credit card?
➡️ Predicted intent: card_arrival

❓ Query: Can I increase my overdraft limit?
➡️ Predicted intent: top_up_limits

❓ Query: I'd like to cancel a payment
➡️ Predicted intent: cancel_transfer

❓ Query: Tell me your working hours
➡️ Predicted intent: card_not_working

